Con este set up: 0.76794 puntos en kaggle

In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [161]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [162]:
# Missing value in test
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [163]:
train['Sex'] = train['Sex'].map({"male": 0, "female": 1})
train['Embarked'] = train['Embarked'].map({"C": 0, "Q": 1, "S": 2})

test['Sex'] = test['Sex'].map({"male": 0, "female": 1})
test['Embarked'] = test['Embarked'].map({"C": 0, "Q": 1, "S": 2})

In [164]:
passengerid_test = test['PassengerId'].values.tolist()

In [165]:
train.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [166]:
train_fe = train.copy(deep=True)
test_fe = test.copy(deep=True)

In [167]:
train_fe.drop(columns=['Age'], inplace=True)
test_fe.drop(columns=['Age'], inplace=True)

In [168]:
def remove_outliers_iqr(df, columns):
    for column in columns: 
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

In [169]:
train_fe = remove_outliers_iqr(train_fe, ['Fare'])

In [170]:
scaler = MinMaxScaler()

In [171]:
# Min_max to normalize the quantitative variables (continuous and discrete)
train_fe[['Fare', 'SibSp', 'Parch']] = scaler.fit_transform(train_fe[['Fare', 'SibSp', 'Parch']])

test_fe[['Fare', 'SibSp', 'Parch']] = scaler.fit_transform(test_fe[['Fare', 'SibSp', 'Parch']])

In [172]:
train_fe = pd.get_dummies(train_fe, columns=['Pclass', 'Embarked'])

test_fe = pd.get_dummies(test_fe, columns=['Pclass', 'Embarked'])

In [173]:
# Change boolean columns to integers
bool_cols = train_fe.select_dtypes(include='bool').columns
train_fe[bool_cols] = train_fe[bool_cols].astype(int)

bool_cols = test_fe.select_dtypes(include='bool').columns
test_fe[bool_cols] = test_fe[bool_cols].astype(int)

In [174]:
train_fe = train_fe.astype(float)

test_fe = test_fe.astype(float)

In [175]:
train_fe.head(10)

,Survived,Sex,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_0.0,Embarked_1.0,Embarked_2.0
0,0.0,0.0,0.2,0.000000,0.111538,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,1.0,0.0,0.000000,0.121923,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,1.0,0.2,0.000000,0.816923,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.000000,0.123846,0.0,0.0,1.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.000000,0.130128,0.0,0.0,1.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.000000,0.797885,1.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.6,0.166667,0.324231,0.0,0.0,1.0,0.0,0.0,1.0
8,1.0,1.0,0.0,0.333333,0.171282,0.0,0.0,1.0,0.0,0.0,1.0
9,1.0,1.0,0.2,0.000000,0.462628,0.0,1.0,0.0,1.0,0.0,0.0
10,1.0,1.0,0.2,0.166667,0.256923,0.0,0.0,1.0,0.0,0.0,1.0


In [176]:
test_fe.head(10)

,Sex,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_0,Embarked_1,Embarked_2
0,0.0,0.000,0.000000,0.015282,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.125,0.000000,0.013663,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.000,0.000000,0.018909,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.000,0.000000,0.016908,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.125,0.111111,0.023984,0.0,0.0,1.0,0.0,0.0,1.0
5,0.0,0.000,0.000000,0.018006,0.0,0.0,1.0,0.0,0.0,1.0
6,1.0,0.000,0.000000,0.014891,0.0,0.0,1.0,0.0,1.0,0.0
7,0.0,0.125,0.111111,0.056604,0.0,1.0,0.0,0.0,0.0,1.0
8,1.0,0.000,0.000000,0.014110,0.0,0.0,1.0,1.0,0.0,0.0
9,0.0,0.250,0.000000,0.047138,0.0,0.0,1.0,0.0,0.0,1.0


In [177]:
X = train_fe.drop('Survived', axis=1).to_numpy() # Features
y = train_fe['Survived'].to_numpy() # Labels

In [178]:
# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)  # use long for classification with nn.CrossEntropyLoss

# Wrap into a TensorDataset
# Create a DataLoader
train_dataloader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=32, shuffle=True)

In [179]:
# Neural Network Definition
class TitanicNN(nn.Module):
    def __init__(self, input_size, hidden_size1=64, hidden_size2=32, output_size=1, dropout_rate=0.3):
        super(TitanicNN, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x
    
# Training Function
def train_model(model, train_loader, criterion, optimizer, num_epochs=100):
    model.train()
    train_losses = []
    train_accuracies = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        for batch_X, batch_y in train_loader:
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_X)
            batch_y = batch_y.unsqueeze(1)
            loss = criterion(outputs, batch_y)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total_predictions += batch_y.size(0)
            correct_predictions += (predicted == batch_y).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct_predictions / total_predictions
        
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
    
    return train_losses, train_accuracies

In [180]:
# Initialize model
input_size = X.shape[1]
model = TitanicNN(input_size=input_size)

In [181]:
# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

print("Starting training...")

# Train the model
train_losses, train_accuracies = train_model(model, train_dataloader, criterion, optimizer, num_epochs=300)

Starting training...
Epoch [20/300], Loss: 0.4591, Accuracy: 0.8052
Epoch [40/300], Loss: 0.4511, Accuracy: 0.7910
Epoch [60/300], Loss: 0.4513, Accuracy: 0.8026
Epoch [80/300], Loss: 0.4313, Accuracy: 0.8116
Epoch [100/300], Loss: 0.4668, Accuracy: 0.7987
Epoch [120/300], Loss: 0.4467, Accuracy: 0.8116
Epoch [140/300], Loss: 0.4207, Accuracy: 0.8052
Epoch [160/300], Loss: 0.4407, Accuracy: 0.8026
Epoch [180/300], Loss: 0.4211, Accuracy: 0.8142
Epoch [200/300], Loss: 0.4351, Accuracy: 0.8116
Epoch [220/300], Loss: 0.4174, Accuracy: 0.8077
Epoch [240/300], Loss: 0.4243, Accuracy: 0.8103
Epoch [260/300], Loss: 0.4377, Accuracy: 0.8219
Epoch [280/300], Loss: 0.3916, Accuracy: 0.8284
Epoch [300/300], Loss: 0.4158, Accuracy: 0.8194


In [182]:
outputs = model(torch.tensor(test_fe.to_numpy(), dtype=torch.float32))

In [183]:
outputs = [int(round(x, 0)) for x in np.array(outputs.tolist()).flatten().tolist()] 

In [184]:
df_output = pd.DataFrame({'PassengerId': passengerid_test ,'Survived': outputs})

In [185]:
df_output.to_csv('output.csv', index=False)